In [2]:
import definitions as df
import data_manipulation as dm
import norm_def as nd
import torch
import matplotlib.pyplot as plt
import model as md
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
import sql_def as sql
import os

In [ ]:
# path_to_database = r"./data"
# path_to_code = r"./"
# os.chdir(path_to_database)
# track_from_sql =  sql.data_from_sql("database.db", "SELECT * FROM TRACKMATEDATA")
# os.chdir(path_to_code)
# track_with_frame = df.separate_trajectories(track_from_sql)
# trajectories1 = df.separate_data(track_with_frame, False)
# trajectories = []
# for temp in trajectories1:
#     f2 = []
#     c = 1
#     for t1 in temp:
#         f2.append(t1)
#         c += 1
#         if c == 10:
#             break
#     trajectories.append(f2)
# trajectories

In [3]:
total_size = 120000
max_x = 1000
max_y = 1000
min_x = 0
min_y = 0
track_len = 20

trajectories = dm.create_synthetic(total_size, max_x, max_y, min_x, min_y, track_len)

train_norm, val_norm, test_norm = df.split_data(trajectories, train_percent=70, val_percent=20)
train, min_x, min_y, range_x, range_y = nd.normalize_data(train_norm)
val, _, _, _, _ = nd.normalize_data(val_norm, min_x, min_y, range_x, range_y)
test, _, _, _, _ = nd.normalize_data(test_norm, min_x, min_y, range_x, range_y)

# Mask the point at index 6
train_data, train_masked_point = df.mask_point_at_index(train, 6)
val_data, val_masked_point = df.mask_point_at_index(val, 6)
test_data, test_masked_point = df.mask_point_at_index(test, 6)

# Prepare the data tensors
src_data_tensor, tgt_data_tensor, src_masks_tensor = df.prepare_data_for_transformer(train_data, train_masked_point)
val_src_data_tensor, val_tgt_data_tensor, val_src_masks_tensor = df.prepare_data_for_transformer(val_data, val_masked_point)
test_src_data_tensor, test_tgt_data_tensor, test_src_masks_tensor = df.prepare_data_for_transformer(test_data, test_masked_point)

hello
hello
hello


In [4]:
def calculate_accuracy_with_tolerance(data, tg, bp, tolerance=10):
    """
    Calculate the accuracy of predictions in data against the target (tg),
    only considering the positions marked as True in the blueprint (bp).
    A prediction is considered correct if it is within 'tolerance' of the target.

    Parameters:
        data (list of lists): The predicted data with predictions injected at masked positions.
        tg (list of lists): The target data to compare against, corresponding only to True mask positions.
        bp (list of lists of bool): Mask indicating where predictions were injected.
        tolerance (float): The tolerance level within which predictions are considered correct.

    Returns:
        float: The accuracy as a percentage of correct predictions.
    """
    correct = 0
    total = 0

    for i, (seq, mask_seq, target_seq) in enumerate(zip(data, bp, tg)):
        target_index = 0
        for j, (mask, pred) in enumerate(zip(mask_seq, seq)):
            if mask:
                if target_index < len(target_seq):
                    target = target_seq
                    total += 1
                    if abs(pred[0] - target[0]) <= tolerance and abs(pred[1] - target[1]) <= tolerance:
                        correct += 1
                    target_index += 1
                else:
                    print("Warning")
    if total == 0:
        return 0  # Avoid division by zero
    accuracy = (correct / total) * 100
    return accuracy

In [5]:
def create_simple_edge_index(num_points):
    row = []
    col = []
    for i in range(num_points):
        if i > 0:
            row.append(i)
            col.append(i - 1)
        if i < num_points - 1:
            row.append(i)
            col.append(i + 1)
    edge_index = torch.tensor([row, col], dtype=torch.long)
    return edge_index

# Example for 20 points
edge_index = create_simple_edge_index(20)
print(edge_index)

tensor([[ 0,  1,  1,  2,  2,  3,  3,  4,  4,  5,  5,  6,  6,  7,  7,  8,  8,  9,
          9, 10, 10, 11, 11, 12, 12, 13, 13, 14, 14, 15, 15, 16, 16, 17, 17, 18,
         18, 19],
        [ 1,  0,  2,  1,  3,  2,  4,  3,  5,  4,  6,  5,  7,  6,  8,  7,  9,  8,
         10,  9, 11, 10, 12, 11, 13, 12, 14, 13, 15, 14, 16, 15, 17, 16, 18, 17,
         19, 18]])


In [6]:
x = nd.unnormalize_point((0.13, 0.14), min_x, min_y, range_x, range_y)
nd.unnormalize_point((0,0),  min_x, min_y, range_x, range_y)

(-185.509165, -184.739612)

In [ ]:
# import torch.optim.lr_scheduler as lr_scheduler


# src_data_tensor, tgt_data_tensor, src_masks_tensor = df.prepare_data_for_transformer(train_data, train_masked_point)

# ## DEBUG


# input_dim = 2
# hidden_dim = 64                                                                                   
# output_dim = 2
# rate = 0.0001
# val_loss_values = []
# #model = md.SelfSupervisedModel(in_channels=3, out_channels=128, input_dim=128, model_dim=256, num_heads=8, num_layers=6, output_dim=3)
# model = md.LSTMModel(2, 32, 2, 1)
# loss_function = nn.MSELoss()
# optimizer = torch.optim.Adam(model.parameters(), lr=rate) ## w
# scheduler = lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=5, min_lr=1e-6, verbose=True)


# dataset = TensorDataset(src_data_tensor, tgt_data_tensor, src_masks_tensor)
# dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

# val_dataset = TensorDataset(val_src_data_tensor, val_tgt_data_tensor, val_src_masks_tensor)
# val_dataloader = DataLoader(val_dataset, batch_size=32, shuffle=False)

# ## TEMP
# n_iters = 40
# batch_size = 32


# loss_values = []
# accuracy_values = []

# for epoch in range(n_iters):
#     total_loss = 0
#     model.train()
#     for src, tgt, mask in dataloader:
#         optimizer.zero_grad()
#         output = model(src)
#         masked_output = output  # Directly use the output for loss calculation
#         loss = loss_function(masked_output, tgt)
#         loss.backward()
#         nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)  # Apply gradient clipping
#         optimizer.step()
#         total_loss += loss.item()
    
#     avg_loss = total_loss / len(dataloader)
#     loss_values.append(avg_loss)

#     # Validation phase
#     model.eval()
#     with torch.no_grad():
#         val_total_loss = 0
#         for src, tgt, mask in val_dataloader:
#             output = model(src)
#             masked_output = output
#             val_loss = loss_function(masked_output, tgt)
#             val_total_loss += val_loss.item()
#         avg_val_loss = val_total_loss / len(val_dataloader)
#         val_loss_values.append(avg_val_loss)
#         print(f'Epoch {epoch+1}/{n_iters}, Validation Loss: {avg_val_loss}, Training Loss: {avg_loss}')

#     # Step the learning rate scheduler
#     scheduler.step(avg_val_loss)

#     # Evaluation phase

        
# with torch.no_grad():
#     predictions = model(test_src_data_tensor)
#     data = test_src_data_tensor.tolist()
#     bp = test_src_masks_tensor.tolist()
#     tg = test_tgt_data_tensor.tolist()
#     data1 = nd.unnormalize_data(data, min_x, min_y, range_x, range_y)
#     tg = nd.unnormalize_points_list(tg, min_x, min_y, range_x, range_y)
#     for i, (seq, mask_seq) in enumerate(zip(data, bp)):
#         for j, mask in enumerate(mask_seq):
#             if mask:
#                 if data1[i][j] == nd.unnormalize_point((0, 0), min_x, min_y, range_x, range_y):
#                     data1[i][j] = nd.unnormalize_point(list(predictions[i].tolist()), min_x, min_y, range_x, range_y)
#                     data[i][j] = list(predictions[i].tolist())
#     acc = calculate_accuracy_with_tolerance(data1, tg, bp, 5)
#     accuracy_values.append(acc)
#     print(f'Accuracy after Epoch {epoch+1}/{n_iters}: {acc:.2f}%')
# # Plotting loss and accuracy
# epochs = range(1, n_iters + 1)

# plt.figure(figsize=(12, 5))

# # Plot loss
# plt.subplot(1, 2, 1)
# plt.plot(epochs, loss_values, 'b', label='Loss')
# plt.xlabel('Epochs')
# plt.ylabel('Loss')
# plt.title('Training Loss')
# plt.legend()

In [7]:
import torch.optim.lr_scheduler as lr_scheduler


src_data_tensor, tgt_data_tensor, src_masks_tensor = df.prepare_data_for_transformer(train_data, train_masked_point)
in_channels = 2  # Number of features in GNN input (x, y)
hidden_channels = 64  # Hidden layer dimension in GNN
out_channels = 128  # Output dimension from GNN
model_dim = 256  # Dimension for transformer model
num_heads = 8  # Number of attention heads in transformer
num_layers = 6  # Number of layers in transformer
output_dim = 2  # Output dimension (x, y)
learning_rate = 0.001

model = md.SelfSupervisedModel(in_channels, hidden_channels, out_channels, model_dim, num_heads, num_layers, output_dim)
loss_function = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
scheduler = lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=5, min_lr=1e-6, verbose=True)

# Create DataLoaders
train_dataset = TensorDataset(src_data_tensor, tgt_data_tensor, src_masks_tensor)
train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)

val_dataset = TensorDataset(val_src_data_tensor, val_tgt_data_tensor, val_src_masks_tensor)
val_dataloader = DataLoader(val_dataset, batch_size=32, shuffle=False)

# Create edge_index for 20 points (fully connected)
edge_index = create_simple_edge_index(20)

# Training loop
n_iters = 20
loss_values = []
val_loss_values = []

for epoch in range(n_iters):
    model.train()
    total_loss = 0
    for src, tgt, mask in train_dataloader:
        optimizer.zero_grad()
        output = model(src, edge_index)
        loss = loss_function(output, tgt)
        loss.backward()
        nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()
        total_loss += loss.item()
    
    avg_loss = total_loss / len(train_dataloader)
    loss_values.append(avg_loss)
    
    # Validation phase
    model.eval()
    val_total_loss = 0
    with torch.no_grad():
        for src, tgt, mask in val_dataloader:
            output = model(src, edge_index)
            val_loss = loss_function(output, tgt)
            val_total_loss += val_loss.item()
    
    avg_val_loss = val_total_loss / len(val_dataloader)
    val_loss_values.append(avg_val_loss)
    
    print(f'Epoch {epoch+1}/{n_iters}, Training Loss: {avg_loss:.4f}, Validation Loss: {avg_val_loss:.4f}')
    scheduler.step(avg_val_loss)

# Evaluation phase
model.eval()
with torch.no_grad():
    predictions = model(test_src_data_tensor, edge_index)
    print("Predictions Shape:", predictions.shape)

    data = test_src_data_tensor.tolist()
    print("Test Data Shape:", test_src_data_tensor.shape)
    bp = test_src_masks_tensor.tolist()
    tg = test_tgt_data_tensor.tolist()

    data = nd.unnormalize_data(data, min_x, min_y, range_x, range_y)
    tg = nd.unnormalize_points_list(tg, min_x, min_y, range_x, range_y)
    
    for i, (seq, mask_seq) in enumerate(zip(data, bp)):
        for j, mask in enumerate(mask_seq):
            if mask:
                if data[i][j] == nd.unnormalize_point((0, 0), min_x, min_y, range_x, range_y):
                    data[i][j] = nd.unnormalize_point(predictions[i].tolist(), min_x, min_y, range_x, range_y)

    acc = calculate_accuracy_with_tolerance(data, tg, bp, 40)
    print(f'Accuracy after Epoch {n_iters}: {acc:.2f}%')

# Plotting loss
epochs = range(1, n_iters + 1)

plt.figure(figsize=(12, 5))

# Plot training loss
plt.subplot(1, 2, 1)
plt.plot(epochs, loss_values, 'b', label='Training Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Training Loss')
plt.legend()

# Plot validation loss
plt.subplot(1, 2, 2)
plt.plot(epochs, val_loss_values, 'r', label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Validation Loss')
plt.legend()

plt.tight_layout()
plt.show()

hello


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/torch/nn/modules/transformer.py:306: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/torch/optim/lr_scheduler.py:28: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn("The verbose parameter is deprecated. Please use get_last_lr() "


Epoch 1/20, Training Loss: 0.0554, Validation Loss: 0.0449
Epoch 2/20, Training Loss: 0.0456, Validation Loss: 0.0468


KeyboardInterrupt: 

In [ ]:
def predict_single_trajectory(model, trajectory):
    """
    Predicts the output for a single trajectory using the trained model.
    
    Parameters:
        model (torch.nn.Module): The trained model.
        trajectory (any): The input trajectory data formatted as required by the model.
        device (str): The device to use ('cpu' or 'cuda').
    
    Returns:
        Any: The prediction made by the model.
    """
    model.eval()
    with torch.no_grad():
        predictions = model(src_data_tensor)
    return predictions

In [ ]:
print("Data", src_data_tensor)
print("Prediction:", prediction)
print("red", src_masks_tensor)


In [ ]:
data = src_data_tensor.tolist()
bp = src_masks_tensor.tolist()
tg = tgt_data_tensor.tolist()
for i, (seq, mask_seq) in enumerate(zip(data, bp)):
    for j, mask in enumerate(mask_seq):
        if mask:
            if data[i][j] == [0.0, 0.0]:
                data[i][j] = list(prediction[i].tolist())

In [ ]:
line = nd.unnormalize_points_list(data[300], min_x, min_y, range_x, range_y)

In [ ]:
x = [point[0] for point in line]
y = [point[1] for point in line]
print(x, y)
fig, ax = plt.subplots()
ax.plot(x, y, label='Data Line', marker='o')
ax.set_xlabel('X Coordinates')
ax.set_ylabel('Y Coordinates')
ax.set_title('Plot of Data Points')
ax.legend()
ax.set_xlim([0, 1050])
ax.set_ylim([0, 1050])
plt.show()

In [ ]:
trajectories = dm.create_synthetic(1, 1000, 1000, 0, 0, 20)
train, min_x, min_y, range_x, range_y = nd.normalize_data(trajectories, min_x, min_y, range_x, range_y)
train_data, train_masked_point = df.mask_point_at_index(train, 6)
src_data_tensor, tgt_data_tensor, src_masks_tensor = df.prepare_data_for_transformer(test_data, test_masked_point)
prediction = predict_single_trajectory(model, src_data_tensor)
print(predictions.shape)
data = src_data_tensor.tolist()
bp = src_masks_tensor.tolist()
tg = tgt_data_tensor.tolist()
data1 = nd.unnormalize_data(data, min_x, min_y, range_x, range_y)
tg = nd.unnormalize_points_list(tg, min_x, min_y, range_x, range_y)
for i, (seq, mask_seq) in enumerate(zip(data, bp)):
    for j, mask in enumerate(mask_seq):
        if mask:
            if data1[i][j] == nd.unnormalize_point((0,0),  min_x, min_y, range_x, range_y):
                data1[i][j] = nd.unnormalize_point(list(prediction[i].tolist()),min_x, min_y, range_x, range_y)
                data[i][j] = list(prediction[i].tolist())
acc = calculate_accuracy_with_tolerance(data1, tg, bp, 20)
print("Accuracy:", acc)